#### Testing combinations with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now)

In [13]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [14]:
# Tha generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 256
base_arr = numpy.random.randint(10, size=base_len)

In [15]:
cumul_arr = numpy.cumsum(base_arr)

In [16]:
start = cumul_arr[:-1]
stop = cumul_arr[1:]

In [17]:
start[:5]

array([ 3,  7,  9, 17, 26])

In [18]:
stop[:5]

array([ 7,  9, 17, 26, 26])

In [19]:
lengths = stop-start
cumul_lengths = numpy.cumsum(lengths)
pairs_lengths = lengths*lengths
pairs_lengths = pairs_lengths.astype(numpy.int32)
lengths_arr = numpy.array([base_len]).astype(numpy.int32)

In [20]:
max(pairs_lengths)

81

In [21]:
left = numpy.zeros(sum(pairs_lengths).astype(numpy.int32))
right = numpy.zeros(sum(pairs_lengths).astype(numpy.int32))

In [22]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start,int* stop,int* lengths,int* pairs_lengths)
{
    int i = threadIdx.x + blockIdx.x*blockDim.x;
    int j = threadIdx.y + blockIdx.y*blockDim.y;
    int k = threadIdx.z + blockIdx.z*blockDim.z;
    
    if (i<lengths[0] && j<pairs_lengths[i] && k<pairs_lengths[i])
    {
        left[i*lengths[0]+j*pairs_lengths[i]+k] = j + start[i];
        right[i*lengths[0]+j*pairs_lengths[i]+k] = k + start[i];
    }
}
''')

In [23]:
func = mod.get_function("comb_events")

In [24]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(stop), cuda.In(lengths_arr),
    cuda.In(pairs_lengths), block=(256, 81, 81), grid=(1,1))

LogicError: cuFuncSetBlockShape failed: invalid argument